# Cryptocurrency Analytics

## Import Libraries

In [2]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import networkx as nx
from pyvis import network as net
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup
import time
from IPython.display import display
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings("ignore")


## Graph Network

### Prepare Data

In [3]:
# Load data from balances
network_data=pd.read_csv('C:/Users/JCRG/Desktop/Master/Dissertation/Data/Transactional_Network_Ether.csv')
network_data.head()

,from_address,to_address,block_timestamp,value_ether
0,0x7ac34681f6aaeb691e150c43ee494177c0e2c183,0x7fb6fefe6c524d9cb06025583c0ad59aad2f6f5b,2018-07-11 11:57:20 UTC,1.00000
1,0xf1cf296c11d3b010e6dd3d48b6c5aaed613e96f1,0x2ec2c8729da460eb9743e86c3d16713f6aa47325,2018-07-11 01:53:49 UTC,3.04768
2,0x8e04af7f7c76daa9ab429b1340e0327b5b835748,0x0000000000085d4780b73119b644ae5ecd22b376,2019-08-20 14:27:26 UTC,0.00000
3,0x8f6147a13514d7cc7c1c85bc4fad8f7f1bc63b57,0x0d8775f648430679a709e98d2b0cb6250d2887ef,2019-05-30 03:04:18 UTC,0.00000
4,0x7b6f730249555a85d39327b90f0080e9dc63daff,0x1cb9ede1a43ab0265d6c44be4f3db554abf822c5,2019-05-30 13:17:56 UTC,250.00000


In [4]:
# Prepare data for network. Group by sender and receiver and count the number of transactions
network_data_grouped=network_data[['from_address','to_address','value_ether']].groupby(['from_address','to_address']).count()
network_data_grouped.reset_index(inplace=True)
network_data_grouped.sort_values(by=['value_ether'],ascending=False,inplace=True)
network_data_grouped.head(10)

,from_address,to_address,value_ether
1749503,0xb8001c3ec9aa1985f6c747e25c28324e4a361ec1,0xdac17f958d2ee523a2206206994597c13d831ec7,185131
378832,0x292f04a44506c2fd49bac032e1ca148c35a478c8,0xdac17f958d2ee523a2206206994597c13d831ec7,107860
1098555,0x7abe0ce388281d2acf297cb089caef3819b13448,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,84100
657758,0x6871eacd33fbcfe585009ab64f0795d7152dc5a0,0xdac17f958d2ee523a2206206994597c13d831ec7,76161
965017,0x6dfc34609a05bc22319fa4cce1d1e2929548c0d7,0xdac17f958d2ee523a2206206994597c13d831ec7,69166
1107158,0x7abe0ce388281d2acf297cb089caef3819b13448,0xdac17f958d2ee523a2206206994597c13d831ec7,62866
1593764,0xa9bff538a906154c80a8dbccd229f3deddfa52d6,0xdac17f958d2ee523a2206206994597c13d831ec7,60683
1839002,0xc5a8859c44ac8aa2169afacf45b87c08593bec10,0x4fabb145d64652a948d72533023f6e7a623c7c53,56508
53855,0x0639556f03714a74a5feeaf5736a4a64ff70d206,0xdac17f958d2ee523a2206206994597c13d831ec7,55257
1839923,0xc5a8859c44ac8aa2169afacf45b87c08593bec10,0x8e870d67f660d95d5be530380d0ec0bd388289e1,35520


### Generate Dataframe Output

In [5]:
# Create object with network data
G = nx.from_pandas_edgelist(network_data_grouped,
                            source='from_address',
                            target='to_address',
                            create_using=nx.DiGraph)

In [6]:
# Get the nodes where the top 10,000 whales are transacting with.
df=pd.DataFrame(G.in_degree()).sort_values(1,ascending=False)
df.columns=['Address','Count']
# Select the top 100 addresses
df=df[:99]

In [7]:
# Get nametag with the addresses from Etherscan through crawler. This is perform to identify easier which are the names of the addresses, if available.
def get_nametag(address):
    #Make API request to Etherscan
    url = 'https://etherscan.io/address/%s'%(address) # Initiate with the first page of accounts with 100 records
    req = Request(url,headers={'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'})   # Etherscan restricts the access for Webscrapping, reason why headers needs to be stated
    response = urlopen(req, timeout=500).read() # Read response
    page_soup = soup(response, "html.parser") # Parse HTML response
    page_soup.find('title')
    a=page_soup.find('title')
    a=str(a)
    a=((a.split('|')[0])[10:])[:-1]
    time.sleep(2) # Etherscan restricts the access if the requests are perform too fast. The function delays the code by two seconds
    return a

In [ ]:
# Create column with nametag
df['tag'] = df['Address'].apply(get_nametag)
# Visualize dataframe
with pd.option_context('display.max_rows', 100, 'display.max_columns', 10):
    display(df)

## Insider Trading Preparation

### Import and preprocess Listing Dates

In [10]:
# Import Listings from Binance
listings=pd.read_csv('Listing_Dates.csv')
listings=df.merge(listings,on='tag',how='left')
listings.columns=['to_address','count','tag','listing_date']

In [11]:
# Merge data with the transactional data from addresses and filter out the addresses that are not in the listings
df2=network_data.merge(listings,on='to_address')
df2=df2[(df2['listing_date']!='Not listed')&(~df2['listing_date'].isna())]
# Configure time format for listing date
df2['listing_date']=pd.to_datetime(df2['listing_date'],infer_datetime_format=True)
# Configure time format for block_timestamp date
df2['block_timestamp']=pd.to_datetime(df2['block_timestamp'],infer_datetime_format=True)
df2['block_timestamp']=df2['block_timestamp'].dt.date
df2['block_timestamp']=pd.to_datetime(df2['block_timestamp'],infer_datetime_format=True)

### Create Filters in Transactional data with Listings In Binance

In [12]:
# Create column with True & False for the difference between the listing date and the block timestamp. This are the transactions that has been done before 30 days of the listing in Binance.
df2['date_before_listing']=(df2['block_timestamp'] < (df2['listing_date']-timedelta(days=30)))
# Create column with True & False for transactions that has been done between the two stated dates, which are between the listing date and 30 days before it.
df2['date_between_listing']=(df2['block_timestamp'] > (df2['listing_date']-timedelta(days=30))) & (df2['block_timestamp'] < df2['listing_date'])
# Create column with key of address and tag for future filtering
df2['key']=df2[['from_address','tag']].agg('-'.join,axis=1)

In [13]:
# Create dataframe with 'from_address' and 'to_address' to filter 'from_address' that have had transactions 30 days before the listing in Binance.
previous_transactions_true=df2[df2['date_before_listing']==True].groupby(['from_address','tag']).nunique()
previous_transactions_true=previous_transactions_true.reset_index()
previous_transactions_true=previous_transactions_true[['from_address','tag']]
# Create column with key of address and tag for future filtering
previous_transactions_true['key']=previous_transactions_true[['from_address','tag']].agg('-'.join,axis=1)

In [14]:
# Create dataframe with filter of transactional data and listings
df3 = df2[~df2.key.isin(list(previous_transactions_true['key']))]
df3=df3[df3['date_between_listing']==True]
df3.groupby(['from_address','tag']).nunique()
# Print
df3

,from_address,to_address,block_timestamp,value_ether,count,tag,listing_date,date_before_listing,date_between_listing,key
7685,0x70cd5f374f496b299e7b3164eba4b32568a5cb79,0x0d8775f648430679a709e98d2b0cb6250d2887ef,2019-03-02,0.0,278,Basic Attention: BAT Token,2019-03-04,False,True,0x70cd5f374f496b299e7b3164eba4b32568a5cb79-Bas...
1184706,0xa205fd7344656c72fdc645b72faf5a3de0b3e825,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,2022-06-15,0.0,311,Wrapped BTC: WBTC Token,2022-06-24,False,True,0xa205fd7344656c72fdc645b72faf5a3de0b3e825-Wra...
1184708,0xa205fd7344656c72fdc645b72faf5a3de0b3e825,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,2022-06-15,0.0,311,Wrapped BTC: WBTC Token,2022-06-24,False,True,0xa205fd7344656c72fdc645b72faf5a3de0b3e825-Wra...
1184908,0xa205fd7344656c72fdc645b72faf5a3de0b3e825,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,2022-06-15,0.0,311,Wrapped BTC: WBTC Token,2022-06-24,False,True,0xa205fd7344656c72fdc645b72faf5a3de0b3e825-Wra...
1185748,0x3b69035f18e923ad2144f775a6156931f9b15d2d,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,2022-06-21,0.0,311,Wrapped BTC: WBTC Token,2022-06-24,False,True,0x3b69035f18e923ad2144f775a6156931f9b15d2d-Wra...
...,...,...,...,...,...,...,...,...,...,...
1787137,0xb8ba36e591facee901ffd3d5d82df491551ad7ef,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b,2021-12-08,0.0,133,Convex Finance: CVX Token,2021-12-23,False,True,0xb8ba36e591facee901ffd3d5d82df491551ad7ef-Con...
1787144,0xb8ba36e591facee901ffd3d5d82df491551ad7ef,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b,2021-12-22,0.0,133,Convex Finance: CVX Token,2021-12-23,False,True,0xb8ba36e591facee901ffd3d5d82df491551ad7ef-Con...
1787146,0xb8ba36e591facee901ffd3d5d82df491551ad7ef,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b,2021-12-08,0.0,133,Convex Finance: CVX Token,2021-12-23,False,True,0xb8ba36e591facee901ffd3d5d82df491551ad7ef-Con...
1787148,0xb8ba36e591facee901ffd3d5d82df491551ad7ef,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b,2021-12-12,0.0,133,Convex Finance: CVX Token,2021-12-23,False,True,0xb8ba36e591facee901ffd3d5d82df491551ad7ef-Con...


In [67]:
v = df3.groupby(['from_address']).tag.value_counts()
df4=pd.DataFrame(v.groupby(level=0).count().sort_values(ascending=False))
df4=df4[df4['tag']>3]
df4

,tag
from_address,
0x024bcbcaad82e67f721799e259ca60bc7d363419,7
0x0c5a2c72c009252f0e7312f5a1ab87de02be6fbe,6
0xcfc50541c3deaf725ce738ef87ace2ad778ba0c5,5
0x7ac34681f6aaeb691e150c43ee494177c0e2c183,5
0xecb6a3e0e99700b32bb03ba14727d99fe8e538cf,5
0x187e3534f461d7c59a7d6899a983a5305b48f93f,5
0xa4517a2b21f85f6a6a63601cea7fa4e34da92856,4
0x22fa8cc33a42320385cbd3690ed60a021891cb32,4
0x49a2dcc237a65cc1f412ed47e0594602f6141936,4


In [72]:
df5=df4.merge(df3,on='from_address',how='left')
df5

,from_address,tag_x,to_address,block_timestamp,value_ether,count,tag_y,listing_date,date_before_listing,date_between_listing,key
0,0x024bcbcaad82e67f721799e259ca60bc7d363419,7,0x0bc529c00c6401aef6d220be8c6ea1667f6ad93e,2020-07-19,0.0,225,Ygov.finance: YFI Token,2020-08-10,False,True,0x024bcbcaad82e67f721799e259ca60bc7d363419-Ygo...
1,0x024bcbcaad82e67f721799e259ca60bc7d363419,7,0x0bc529c00c6401aef6d220be8c6ea1667f6ad93e,2020-07-21,0.0,225,Ygov.finance: YFI Token,2020-08-10,False,True,0x024bcbcaad82e67f721799e259ca60bc7d363419-Ygo...
2,0x024bcbcaad82e67f721799e259ca60bc7d363419,7,0x0bc529c00c6401aef6d220be8c6ea1667f6ad93e,2020-07-19,0.0,225,Ygov.finance: YFI Token,2020-08-10,False,True,0x024bcbcaad82e67f721799e259ca60bc7d363419-Ygo...
3,0x024bcbcaad82e67f721799e259ca60bc7d363419,7,0x0bc529c00c6401aef6d220be8c6ea1667f6ad93e,2020-08-03,0.0,225,Ygov.finance: YFI Token,2020-08-10,False,True,0x024bcbcaad82e67f721799e259ca60bc7d363419-Ygo...
4,0x024bcbcaad82e67f721799e259ca60bc7d363419,7,0x0bc529c00c6401aef6d220be8c6ea1667f6ad93e,2020-08-03,0.0,225,Ygov.finance: YFI Token,2020-08-10,False,True,0x024bcbcaad82e67f721799e259ca60bc7d363419-Ygo...
...,...,...,...,...,...,...,...,...,...,...,...
155,0xe2e4f2a725e42d0f0ef6291f46c430f963482001,4,0x0bc529c00c6401aef6d220be8c6ea1667f6ad93e,2020-07-19,0.0,225,Ygov.finance: YFI Token,2020-08-10,False,True,0xe2e4f2a725e42d0f0ef6291f46c430f963482001-Ygo...
156,0xe2e4f2a725e42d0f0ef6291f46c430f963482001,4,0xd533a949740bb3306d119cc777fa900ba034cd52,2020-08-14,0.0,268,Curve.fi: CRV Token,2020-08-15,False,True,0xe2e4f2a725e42d0f0ef6291f46c430f963482001-Cur...
157,0xe2e4f2a725e42d0f0ef6291f46c430f963482001,4,0xd533a949740bb3306d119cc777fa900ba034cd52,2020-08-14,0.0,268,Curve.fi: CRV Token,2020-08-15,False,True,0xe2e4f2a725e42d0f0ef6291f46c430f963482001-Cur...
158,0xe2e4f2a725e42d0f0ef6291f46c430f963482001,4,0x04fa0d235c4abf4bcf4787af4cf447de572ef828,2020-08-30,0.0,79,UMA: UMA Token,2020-09-09,False,True,0xe2e4f2a725e42d0f0ef6291f46c430f963482001-UMA...


In [76]:
v = df5.groupby(['from_address']).tag_y.value_counts()
pd.DataFrame(v.groupby(level=1).count().sort_values(ascending=False))


,tag_y
tag_y,
Curve.fi: CRV Token,14
Ygov.finance: YFI Token,14
SushiSwap: SUSHI Token,10
Compound: Comptroller,9
Balancer: BAL Token,5
Aave: AAVE Token,4
UMA: UMA Token,4
ENS: ENS Token,3
Synthetix: Proxy SNX Token,2


In [48]:
df4.to_csv("Insider_Trading_Addresses.csv")
